In [1]:
import pandas as pd
import numpy as np
# import ot
# import re
# import unicodedata
# from bs4 import BeautifulSoup
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from gensim.models import KeyedVectors, Word2Vec
import gensim.downloader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm

In [2]:
movie_df  = pd.read_csv("movie_data.csv").dropna()

In [3]:
movie_df

,tmdbId,title,clean_overview
0,862,Toy Story,led woody andy toys live happily room andy bir...
1,8844,Jumanji,siblings judy peter discover enchanted board g...
2,15602,Grumpier Old Men,family wedding reignites ancient feud neighbor...
3,31357,Waiting to Exhale,cheated mistreated stepped women holding breat...
4,11862,Father of the Bride Part II,george banks recovered daughter wedding receiv...
...,...,...,...
9622,432131,黒執事 Book of the Atlantic,ciel learns aurora society rumored researching...
9623,445030,ノーゲーム・ノーライフ ゼロ,ancient disboard riku angry young warrior inte...
9624,479308,Flint,woman deals toxic water scandal flint michigan...
9625,483455,文豪ストレイドッグス DEAD APPLE,large scale catastrophe occurring across plane...


In [4]:
corpus = []
for desc in tqdm(movie_df['clean_overview']):
    corpus.append(desc.split())

  0%|          | 0/9618 [00:00<?, ?it/s]

In [5]:
# pretrained_model = KeyedVectors.load('initial_model')
pretrained_model = gensim.downloader.load('word2vec-google-news-300')

In [6]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df = 2, stop_words='english')
tfidf_vectors = tfidf.fit_transform(movie_df['clean_overview'])
tfidf_feature = tfidf.get_feature_names_out()

In [7]:
vectors_df = pd.DataFrame(tfidf_vectors.toarray(), columns = tfidf_feature)

In [8]:
doc_vectors = []
for index, desc in tqdm(enumerate(corpus)):
    weighted_word_vector = np.zeros(300)
    weighted_sum = 0
    for word in desc:
        if word in tfidf_feature and word in pretrained_model:
            weighted_word_vector += pretrained_model[word] * vectors_df.loc[index, word]
            weighted_sum += vectors_df.loc[index, word]
        if weighted_sum != 0:
            weighted_word_vector /= weighted_sum
    doc_vectors.append(weighted_word_vector)

0it [00:00, ?it/s]

In [9]:
cosine_similarities = cosine_similarity(doc_vectors, doc_vectors)

In [10]:
cosine_similarities_ = cosine_similarity(vectors_df, vectors_df)

In [11]:
def recommendation(title):
    indices = pd.Series(movie_df.index, index = movie_df['title']).drop_duplicates()
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    print(sim_scores)
    book_indices = [i[0] for i in sim_scores]
    recommend = movie_df.iloc[book_indices]["title"]
    print(recommend)

In [12]:
recommendation("Mission: Impossible")

[(7778, 0.9987755673810671), (7265, 0.994846548056306), (4086, 0.9850812766645703), (7762, 0.9835303305505907), (2292, 0.969345692097803)]
7782                     Goon
7267               The Losers
4087                    Thief
7766    A Very Potter Musical
2293             Meatballs IV
Name: title, dtype: object
